In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
random_seed = 0
save_to_disk = False
model_prefix = 'nn'
ensemble_logits = True
train_on_logits = True

In [ ]:
target = 'logit' if train_on_logits else 'hard'
ensemble_pred = 'ensemble-logit' if train_on_logits else 'ensemble-hard'

model_name = f'{model_prefix}_{target}_{ensemble_pred}_{random_seed}_distilled'

In [ ]:
from numpy.random import seed
seed(random_seed)
from tensorflow.random import set_seed
set_seed(random_seed)

In [ ]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from typing import List
import glob


In [ ]:
from knowledge_distillation.io import *
from knowledge_distillation.ensemble import UnbiasedAverage
from knowledge_distillation.processing import * 
from knowledge_distillation.nn import *

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Load data

In [ ]:
df = load_adult()

# Preprocessing

In [ ]:
X, y, target_names = scale_fastai(df, target='salary')

X_train, X_test, y_train, y_test = split_with_seed(X, y)

# Load models

In [ ]:
model_paths = sorted(glob.glob(f"{ASSETS_PATH / model_prefix}_*.tf"))

In [ ]:
model_names = [p.split('/')[-1].split('.')[0] for p in model_paths]
model_names

['nn_1',
 'nn_10',
 'nn_2',
 'nn_3',
 'nn_4',
 'nn_5',
 'nn_6',
 'nn_7',
 'nn_8',
 'nn_9']

In [ ]:
# load models
models = [load_keras_classifier(name) for name in model_names]    

In [ ]:
first = models[0]
assert len(first.predict(X_test.head(5))) == 5

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
first.predict_proba(X_test.head(5))[:,1], first.predict(X_test.head(5))

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


(array([0.01872203, 0.14924997, 0.33599758, 0.04015872, 0.05391678],
       dtype=float32),
 array([[0],
        [0],
        [0],
        [0],
        [0]], dtype=int8))

In [ ]:
first.predict_proba(X_test.head(5))[:,1]

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([0.01872203, 0.14924997, 0.33599758, 0.04015872, 0.05391678],
      dtype=float32)

# Evaluate ensemble predictions

In [ ]:
z = first.predict_proba(X_test.head(5))[:,1]

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [ ]:
s = z
s = np.dstack((s, z))
s.reshape((s.shape[0], s.shape[1]*s.shape[2]))
# np.dstack((s, z))

array([[0.01872203, 0.01872203, 0.14924997, 0.14924997, 0.33599758,
        0.33599758, 0.04015872, 0.04015872, 0.05391678, 0.05391678]],
      dtype=float32)

In [ ]:
def stack_predictions(members, inputX, logits=False):
    stack = []
    for model in members:
        # make prediction
        if logits:
            yhat = model.predict_proba(inputX, verbose=0)[:,1].T
        else:
            yhat = model.predict(inputX, verbose=0).T 
                
        # stack predictions into [rows, members, probabilities]
        stack.append(yhat)
        
#     stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return np.vstack(stack).T

In [ ]:
yhat = first.predict(X_train.head(5), verbose=0).T
yhat

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[0, 0, 0, 0, 0]], dtype=int8)

In [ ]:
np.mean(np.vstack([yhat, yhat]).T, axis=1)

array([0., 0., 0., 0., 0.])

In [ ]:
stack_predictions([first, first, first], X_train.head(5), logits=True) 

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([[0.07153332, 0.07153332, 0.07153332],
       [0.30757236, 0.30757236, 0.30757236],
       [0.0201382 , 0.0201382 , 0.0201382 ],
       [0.04635373, 0.04635373, 0.04635373],
       [0.05969843, 0.05969843, 0.05969843]], dtype=float32)

In [ ]:
def ensemble_predict_proba(models, x, logits):
    return np.mean(stack_predictions(models, x, logits=logits), axis=1)

In [ ]:
assert len(ensemble_predict_proba(models, X_train.head(10), logits=True) == 10), "unexpected number of predictions"

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-package

In [ ]:
@dataclass
class TrainedKerasEnsemble():
    keras_estimators:List[KerasClassifier]
    ensemble_logits:bool
    
    def predict_proba(self, X):
        return ensemble_predict_proba(self.keras_estimators, X, logits=self.ensemble_logits)
#         individual_preds = [model.predict_proba(X) for model in self.keras_estimators]
#         individual_preds = np.stack(individual_preds, axis=1)
        
#         return self.final_estimator.predict_proba(individual_preds)
    
    def predict(self, X):
        return ensemble_predict_proba(self.keras_estimators, X, logits=self.ensemble_logits).astype(int)
    
    

In [ ]:
ensemble = TrainedKerasEnsemble(models, ensemble_logits=ensemble_logits)

In [ ]:

evaluate_model(X_train, X_test, y_train, y_test, ensemble, 
               f"{model_prefix}_{'logits' if ensemble_logits else 'hard'}_ensemble", 
               save_to_disk=save_to_disk, target_names=target_names)


/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-package

=== Train ===
              precision    recall  f1-score   support

       <=50K       0.76      1.00      0.87     19778
        >50K       0.99      0.02      0.04      6270

    accuracy                           0.76     26048
   macro avg       0.88      0.51      0.45     26048
weighted avg       0.82      0.76      0.67     26048


=== Test ===
              precision    recall  f1-score   support

       <=50K       0.76      1.00      0.87      4942
        >50K       1.00      0.02      0.05      1571

    accuracy                           0.76      6513
   macro avg       0.88      0.51      0.46      6513
weighted avg       0.82      0.76      0.67      6513




,model_name,data,accuracy,precision,recall,f1,auc
0,nn_logits_ensemble,train,0.763744,0.991525,0.018660,0.036631,0.509305
1,nn_logits_ensemble,test,0.764471,1.000000,0.023552,0.046020,0.511776


# Model distillation: train on ensemble output

In [ ]:
# Create a single NN, identical to the ones in the ensemble
model = KerasClassifier(
    build_fn=create_nn,
    **train_params,
    verbose=1
)


In [ ]:
if train_on_logits:
    y_train_pred_ensemble = ensemble.predict_proba(X_train)
else:
    y_train_pred_ensemble = ensemble.predict(X_train)
y_train_pred_ensemble[:5]

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32

array([0, 0, 0, 0, 0])

In [ ]:
# train the NN on the output of the ensemble
model.fit(X_train, y_train_pred_ensemble)


Epoch 1/15
18/18 [==============================] - 4s 88ms/step - loss: 0.6624 - accuracy: 0.6664 - val_loss: 1.8154 - val_accuracy: 0.0845
Epoch 2/15
18/18 [==============================] - 1s 41ms/step - loss: 0.4416 - accuracy: 0.8952 - val_loss: 1.2374 - val_accuracy: 0.1527
Epoch 3/15
18/18 [==============================] - 1s 55ms/step - loss: 0.3281 - accuracy: 0.9475 - val_loss: 1.1388 - val_accuracy: 0.2237
Epoch 4/15
18/18 [==============================] - 1s 76ms/step - loss: 0.2560 - accuracy: 0.9583 - val_loss: 1.2445 - val_accuracy: 0.2411
Epoch 5/15
18/18 [==============================] - 1s 62ms/step - loss: 0.2079 - accuracy: 0.9632 - val_loss: 0.5464 - val_accuracy: 0.7267
Epoch 6/15
18/18 [==============================] - 1s 74ms/step - loss: 0.1671 - accuracy: 0.9747 - val_loss: 0.4525 - val_accuracy: 0.8366
Epoch 7/15
18/18 [==============================] - 1s 53ms/step - loss: 0.1357 - accuracy: 0.9807 - val_loss: 0.4261 - val_accuracy: 0.8369
Epoch 8/15
18

# Evaluate distilled model

In [ ]:
evaluate_model(X_train, X_test, y_train, y_test, model, model_name, save_to_disk=save_to_disk, target_names=target_names)


/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


1/7 [===>..........................] - ETA: 0s

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


7/7 [==============================] - 0s 8ms/step
=== Train ===
              precision    recall  f1-score   support

       <=50K       0.80      0.99      0.89     19778
        >50K       0.91      0.24      0.38      6270

    accuracy                           0.81     26048
   macro avg       0.86      0.61      0.63     26048
weighted avg       0.83      0.81      0.77     26048


=== Test ===
              precision    recall  f1-score   support

       <=50K       0.80      0.99      0.89      4942
        >50K       0.90      0.24      0.38      1571

    accuracy                           0.81      6513
   macro avg       0.85      0.62      0.63      6513
weighted avg       0.83      0.81      0.76      6513




,model_name,data,accuracy,precision,recall,f1,auc
0,nn_0_distilled,train,0.810926,0.913337,0.237002,0.376345,0.614936
1,nn_0_distilled,test,0.810226,0.903614,0.238701,0.377644,0.615304


In [ ]:
model.predict(X)

17/32 [==============>...............] - ETA: 0s

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


32/32 [==============================] - 0s 6ms/step


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])